### Question 1 - Create Dataframe


In [1]:
#import libraries
import numpy as np
import pandas as pd
import requests
import json
import csv

#install beautiful soup for web scraping
!pip install watson-developer-cloud==1.5
from bs4 import BeautifulSoup

print("Packages installed")


     |████████████████████████████████| 225kB 7.5MB/s eta 0:00:01
     |████████████████████████████████| 880kB 8.7MB/s eta 0:00:01
     |████████████████████████████████| 3.1MB 39.4MB/s eta 0:00:01
     |████████████████████████████████| 2.3MB 40.1MB/s eta 0:00:01
     |████████████████████████████████| 61kB 15.6MB/s eta 0:00:01
     |████████████████████████████████| 174kB 42.2MB/s eta 0:00:01
     |████████████████████████████████| 163kB 43.7MB/s eta 0:00:01
     |████████████████████████████████| 81kB 28.4MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/81/cd/de/4e0916f623c2d125502e493394fd333ed693960264d4b7e524
Successfully built watson-developer-cloud
ERROR: automat 20.2.0 has requirement attrs>=19.2.0, but you'll have attrs 18.2.0 which is incompatible.
  Found existing installation: cryptography 2.5
    Uninstalling cryptography-2.5:
      Successfully uninstalled cryptography-2.5
Packages installed


#### Create the dataframe from Wikipedia

In [2]:
# get url from Wikipedia
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#create a BeautifulSoup object
soup = BeautifulSoup(source, 'lxml')

#create variable for the table of postal codes  
table = soup.find('table')

# Create Pandas Date 
df = pd.DataFrame(columns = ['PostalCode', 'Borough','Neighbourhood'])


# Search all the postcode, borough, neighborhood 
#scan all rows in the first table
for tr in table.find_all('tr'):
    #create empty dataframe
    row_data=[]
    #scan all cells in each row
    for td in tr.find_all('td'):
        #append row to dataframe and strip off non-text info
        row_data.append(td.text.strip())
    #add row from row_data to df when there are 3 columns
    if len(row_data)==3:
        df.loc[len(df)] = row_data


#remove rows with Borough column = 'Not assigned'
df = df[df['Borough'] != 'Not assigned']

df.head(10)


,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


#### Manipulate dataframes to bring boroughs with more than one neighbourhood to the same row. 

In [3]:

#create temporary dataframe and group by postal code
temp_df=df.groupby('PostalCode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))

#reset the index to retain column headings
temp_df=temp_df.reset_index(drop=False)

#Rename the column heading in the original dataframe - will be deleted after merge
df.rename(columns={'Neighbourhood':'Neighbourhood_old'},inplace=True)

#merge df and temp_df on the Postal Code column - the common column between the two dfs
df_merge = pd.merge(df, temp_df, on='PostalCode')

#drop the neighbourhood_old column
df_merge.drop(['Neighbourhood_old'],axis=1,inplace=True)

#drop duplicate entries
df_merge.drop_duplicates(inplace=True)

#display first 11 entries
df_merge.head(11)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Etobicoke,Islington Avenue
7,M1B,Scarborough,"Rouge, Malvern"
9,M3B,North York,Don Mills North
10,M4B,East York,"Woodbine Gardens, Parkview Hill"
12,M5B,Downtown Toronto,"Ryerson, Garden District"


#### Get the shape of the notebook

In [4]:
#shape of notebook
df_merge.shape

(103, 3)